In [1]:
from idlelib.iomenu import encoding

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from sympy import sequence
from unicodedata import category

In [2]:
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()

True

In [3]:
import pandas as pd

books=pd.read_csv('books_cleaned.csv')

In [4]:
books['tagged_description'].to_csv("tagged_description.txt",sep='\n',index=False,header=False)

In [5]:
raw_documents=TextLoader("tagged_description.txt",encoding="utf-8").load()
text_splitter=CharacterTextSplitter(chunk_overlap=0,chunk_size=0,separator="\n")
documents=text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

KeyboardInterrupt: 

In [13]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db_books = Chroma.from_documents(documents, embedding=embedding)


In [12]:
query="A book to teach children about nature"
docs=db_books.similarity_search(query,k=10)
docs


[Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(metadata={'source': 'tagged_description.txt'}, page_content="9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illu

In [24]:
books[books['isbn13']==int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...,Children's Fiction


In [15]:
def retreive_semantic_recommendations(query:str,top_k=10)->pd.DataFrame:
    recs=db_books.similarity_search(query,k=50)
    books_list=[]
    for i in range(0,len(recs)):
        books_list+=[int(recs[i].page_content.strip('"').split()[0])]

    return  books[books['isbn13'].isin(books_list)].head(top_k)

In [20]:
retreive_semantic_recommendations("A book to teach about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
31,9780007105045,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"9780007105045 ""The two works 'On fairy-stories...",NaN
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...,NaN
306,9780060932664,006093266X,Collected Novellas,Gabriel Garcia Marquez,Fiction,http://books.google.com/books/content?id=JRcVu...,"Renowned as a master of magical realism, Gabri...",1999.0,4.01,288.0,822.0,Collected Novellas,9780060932664 Renowned as a master of magical ...,Fiction
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...,Fiction
383,9780061144899,0061144894,When the Heart Waits,Sue Monk Kidd,Religion,http://books.google.com/books/content?id=JlP91...,From the Bestselling Author of The Secret Life...,2006.0,4.17,240.0,2141.0,When the Heart Waits: Spiritual Direction for ...,9780061144899 From the Bestselling Author of T...,Nonfiction
393,9780061238239,0061238236,The End of Days,Zecharia Sitchin,History,http://books.google.com/books/content?id=EIBlj...,A conclusion to the Earth Chronicles series br...,2007.0,4.06,336.0,470.0,The End of Days: Armageddon and Prophecies of ...,9780061238239 A conclusion to the Earth Chroni...,Nonfiction
396,9780062511119,0062511114,A Manual for Living,Epictetus,Religion,http://books.google.com/books/content?id=cofP7...,The essence of perennial Stoic wisdom in aphor...,1994.0,4.22,96.0,148.0,A Manual for Living,9780062511119 The essence of perennial Stoic w...,Nonfiction
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit...",Children's Fiction
991,9780192862099,019286209X,The Origins of Life,John Maynard Smith;Eörs Szathmáry,Science,http://books.google.com/books/content?id=nHDbB...,'I can recommend this book as a thoroughly int...,2000.0,4.11,192.0,41.0,The Origins of Life: From the Birth of Life to...,9780192862099 'I can recommend this book as a ...,Nonfiction
1060,9780231054751,0231054750,Genetics and the Origin of Species,Theodosius Dobzhansky,Science,http://books.google.com/books/content?id=s7igy...,Featuring an introduction by Stephen Jay Gould...,1982.0,4.15,364.0,24.0,Genetics and the Origin of Species,9780231054751 Featuring an introduction by Ste...,Nonfiction


In [16]:
category_mapping={
    'Fiction':"Fiction",
    'Juvenile Fiction': "Children's Fiction" ,
    'Biography & Autobiography': "Nonfiction" ,
    'History': "Nonfiction" ,
    'Literary Criticism': "Nonfiction" ,
    'Philosophy': "Nonfiction" ,
    'Religion': "Nonfiction" ,
    'Comics & Graphic Novels': "Fiction" ,
    'Drama': "Fiction" ,
    'Science' : "Nonfiction" ,
    'Poetry': "Fiction",
    'Juvenile Nonfiction':"Children's Nonfiction"}

In [17]:
books["simple_categories"]=books['categories'].map(category_mapping)

In [18]:
books[~(books['simple_categories'].isna())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Children's Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,Taken at the Flood,9780007121014 A Few Weeks After Marrying An At...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",Fiction
5188,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5189,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [19]:
from transformers import pipeline
fiction_categories=["Fiction","Nonfiction"]

# Load the model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli",device="cpu")

Device set to use cpu


In [21]:
sequence=books.loc[books['simple_categories']=="Fiction","description"].reset_index(drop=True)[0]


In [22]:
classifier(sequence,fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [23]:
import numpy as np
max_index=np.argmax(classifier(sequence,fiction_categories)['scores'])
max_label=classifier(sequence,fiction_categories)["labels"][max_index]
max_label


'Fiction'

In [25]:
def generate_predictions(sequence,fiction_categories):
    predictions=classifier(sequence,fiction_categories)
    max_index=np.argmax(predictions["scores"])
    max_label=predictions["labels"][max_index]
    return max_label

In [26]:
from tqdm import tqdm

actual_categories=[]
predicted_categories=[]

for i in tqdm(range(0,300)):
    sequence=books.loc[books['simple_categories']=="Fiction","description"].reset_index(drop=True)[i]
    predicted_categories+=[generate_predictions(sequence,fiction_categories)]
    actual_categories+=["Fiction"]


100%|██████████| 300/300 [10:25<00:00,  2.09s/it]


In [27]:
for i in tqdm(range(0,300)):
    sequence=books.loc[books['simple_categories']=="Nonfiction","description"].reset_index(drop=True)[i]
    predicted_categories+=[generate_predictions(sequence,fiction_categories)]
    actual_categories+=["Nonfiction"]

100%|██████████| 300/300 [05:14<00:00,  1.05s/it]


In [29]:
import pandas as pd
predictions_df=pd.DataFrame({"actual_category":actual_categories,"predicted_category":predicted_categories})

In [30]:
predictions_df

,actual_category,predicted_category
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction
...,...,...
595,Nonfiction,Nonfiction
596,Nonfiction,Fiction
597,Nonfiction,Nonfiction
598,Nonfiction,Nonfiction


In [31]:
predictions_df["correct_prediction"]=(np.where(predictions_df["actual_category"]==predictions_df["predicted_category"],1,0))

In [32]:
predictions_df["correct_prediction"].sum()/len(predictions_df)

np.float64(0.7783333333333333)

In [33]:
isbns=[]
predicted_cats=[]

missing_cats=books.loc[books['simple_categories'].isna(),["isbn13","description"]].reset_index(drop=True)


In [34]:
for i in tqdm(range(0,len(missing_cats))):
    sequence=missing_cats["description"][i]
    predicted_cats+=[generate_predictions(sequence,fiction_categories)]
    isbns+=[missing_cats["isbn13"][i]]

100%|██████████| 1454/1454 [27:42<00:00,  1.14s/it]


In [35]:
missing_predictions_df=pd.DataFrame({"isbn13":isbns,"predicted_category":predicted_cats})

In [36]:
missing_predictions_df

,isbn13,predicted_category
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1449,9788125026600,Nonfiction
1450,9788171565641,Fiction
1451,9788172235222,Fiction
1452,9788173031014,Nonfiction


In [44]:
books=pd.merge(books,missing_predictions_df,on="isbn13",how="left")
books["simple_categories"]=np.where(books["simple_categories"].isna(),books["predicted_category"],books["simple_categories"])
books=books.drop(columns="predicted_category")
books.to_csv("books_new.csv",index=False)


In [10]:
books=pd.read_csv("books_new.csv")
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories,predicted_category_x,predicted_category_y
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,NaN,NaN
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,Fiction,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,NaN,NaN
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,Nonfiction,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,Nonfiction,Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,Fiction,Fiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,Nonfiction,Nonfiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,Fiction,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,NaN,NaN


In [11]:
books=books.drop(columns=["predicted_category_x","predicted_category_y"])
